# TASKS 

**Write valid SPARQL queries to:**

1. List the top four oldest people in the graph from oldest to youngest
2. Identify the shortest student who has at least four classmates 
3. List the teachers who have the highest salaries (in increasing order) and have at least five colleagues?
4. Return the mean (average) age of male professors, and the mean age of female professors
5. For each employee, calculate their "net salary"\
Net Salary Calculation = Base salary - income tax (%4 of the base salary) - pension (%6 of the base salary)

6. List all people with their first name or family name starting with the letter F, E, or A

7. List all `dbe:Researcher` using inference

Requirements: your query should also returns subclasses of `dbe:Researcher`, such as PhD Candidates and AssistantResearcher, without directly using their URI (use the `rdfs:subClassOf` properties)

8. Create missing relations when `:isSupervisedBy` is defined in one direction but not `:supervises` in the other direction. For example, if john `:isSupervisedBy` mary, then we know that mary `:supervises` john

Requirements: you should use a CONSTRUCT query to build the missing triples

9. Count the number of men and women per `rdf:type` in the graph (aka. the `a` relation)

10. Write a federated query to Wikidata to retrieve Arya Parker  date of establishment the starting date of Maastricht University

11. Open-ended question!  Write a meaningful question and its corresponding SPARQL query using at least a subquery and group by.

In [ ]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 514 kB/s 


#Data

In [ ]:
university_rdf = """@prefix ex: <http://example.org/> .                         
@prefix schema: <http://schema.org/> .                    
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .        
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .   
@prefix wde: <https://www.wikidata.org/wiki/> .           
@prefix wdp: <http://www.wikidata.org/prop/statement/> .
@prefix dbp: <http://dbpedia.org/property/> .  
@prefix dbe: <https://dbpedia.org/page/> .   
@prefix dbo: <https://dbpedia.org/ontology/> .   
@prefix geo: <http://www.geonames.org/ontology#> .        
@prefix wiki: <http://www.wikidata.org/entity/>.
@prefix : <http://example.org/voc#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .


#university

# Classes

dbe:Researcher rdfs:subClassOf schema:Person .
dbe:Teacher rdfs:subClassOf schema:Person .
dbe:Student rdfs:subClassOf schema:Person .

# PhD candidate
wiki:Q12764792 rdfs:subClassOf dbe:Student,dbe:Researcher.

# Professor
wiki:Q121594 rdfs:subClassOf dbe:Researcher, dbe:Teacher .

dbe:AssistantProfessor rdfs:subClassOf wiki:Q121594, dbe:Researcher, dbe:Teacher.

schema:Course a owl:Class .

# Properties
schema:teaches a owl:ObjectProperty ; 
      rdfs:domain dbe:Teacher ;
    	rdfs:range schema:Course .	

:givesLecture rdfs:subPropertyOf schema:teaches .
:givesLab rdfs:subPropertyOf schema:teaches .

schema:title a owl:DatatypeProperty ;
	rdfs:range xsd:string .

:isSupervisedBy a owl:ObjectProperty .
:supervises owl:inverseOf :isSupervisedBy .

schema:baseSalary a owl:DatatypeProperty ;
	rdfs:domain schema:employee.


# Alice
ex:AliceDoe a schema:Person;
    a wiki:Q12764792;
    schema:baseSalary "500"^^xsd:positiveInteger;
    :isSupervisedBy ex:JohnBrown;
    dbp:age "22"^^xsd:positiveInteger;                      
    schema:birthDate "2000-12-16"^^xsd:date;          
    schema:familyName "Doe";
    rdfs:label "Alice Doe";
    schema:gender schema:Female;
    schema:givenName "Alice";
    schema:height "178"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:AryaParker, ex:PatrickWilson;
    ex:classmate ex:KatrinePhill, ex:EricJohnson, ex:SelenaGarcia.


# John
ex:JohnBrown a schema:Person;
    a wiki:Q121594;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "5300"^^xsd:positiveInteger;
    dbp:age "53"^^xsd:positiveInteger;
    schema:birthDate "1969-05-07"^^xsd:date;
    schema:familyName "John";
    rdfs:label "John Brown";
    schema:gender schema:Male;
    schema:givenName "John";
    schema:height "185"^^xsd:positiveInteger;
    schema:colleague ex:AryaParker, ex:FrankJones, ex:johnPitter, ex:codyModly, ex:JinzhouYang, ex:OzgeErten.

# Arya
ex:AryaParker a schema:Person;
    a dbe:AssistantProfessor;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "5000"^^xsd:positiveInteger;
    dbp:age "57"^^xsd:positiveInteger;
    schema:birthDate "1963-03-31"^^xsd:date;
    schema:familyName "Parker";
    rdfs:label "Arya Parker";
    schema:gender schema:Female;
    schema:givenName "Parker";
    schema:height "164"^^xsd:positiveInteger;
    schema:colleague ex:JohnBrown, ex:FrankJones, ex:johnPitter, ex:codyModly, ex:JinzhouYang, ex:OzgeErten.


# Katrine
ex:KatrinePhill a schema:Person;
    a dbe:Researcher;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "65"^^xsd:positiveInteger;
    schema:birthDate "1947-01-14"^^xsd:date;
    schema:familyName "Phill";
    rdfs:label "Katrine Phill";
    schema:gender schema:Female;
    schema:givenName "Katrine";
    schema:height "175"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:FrankJones, ex:PatrickWilson;
    ex:classmate ex:AliceDoe, ex:GeorgeAnderson, ex:EricJohnson, ex:SelenaGarcia, ex:KarenJones.
    

# Frank
ex:FrankJones a schema:Person;
    a dbe:AssistantProfessor;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "4300"^^xsd:positiveInteger;
    dbp:age "76"^^xsd:positiveInteger;
    schema:birthDate "1946-12-23"^^xsd:date;
    schema:familyName "Jones";
    rdfs:label "Frank Jones";
    schema:gender schema:Male;
    schema:givenName "Frank";
    schema:height "174"^^xsd:positiveInteger;
    schema:colleague ex:JohnBrown, ex:AryaParker.

# Karen
ex:KarenJones a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "18"^^xsd:positiveInteger;
    schema:birthDate "2004-06-17"^^xsd:date;
    schema:familyName "Karen";
    rdfs:label "Karen Jones";
    schema:gender schema:Female;
    schema:givenName "Karen";
    schema:height "182"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:AryaParker, ex:FrankJones, ex:PatrickWilson;
    ex:classmate ex:SelenaGarcia,ex:HarryLee, ex:KatrinePhill.

# George
ex:GeorgeAnderson a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "17"^^xsd:positiveInteger;
    schema:birthDate "2005-02-11"^^xsd:date;
    schema:familyName "Anderson";
    rdfs:label "George Anderson";
    schema:gender schema:Male;
    schema:givenName "George";
    schema:height "178"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:AryaParker,ex:FrankJones;
    ex:classmate ex:KatrinePhill, ex:MelanieDavis, ex:HarryLee, ex:ElsaTaylor,ex:HarryLee.

# Selena
ex:SelenaGarcia a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "14"^^xsd:positiveInteger;
    schema:birthDate "2008-12-25"^^xsd:date;
    schema:familyName "Garcia";
    rdfs:label "Selena Garcia";
    schema:gender schema:Female;
    schema:givenName "Garcia";
    schema:height "172"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:PhillipMiller, ex:FrankJones ;
    ex:classmate ex:AliceDoe, ex:MelanieDavis,ex:KatrinePhill.

# Melanie
ex:MelanieDavis a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "19"^^xsd:positiveInteger;
    schema:birthDate "2003-07-18"^^xsd:date;
    schema:familyName "Davis";
    rdfs:label "Melanie Davis";
    schema:gender schema:Female;
    schema:givenName "Melanie";
    schema:height "172"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:PhillipMiller, ex:FrankJones ;
    ex:classmate ex:GeorgeAnderson, ex:SelenaGarcia, ex:HarryLee.


# Eric
ex:EricJohnson a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "16"^^xsd:positiveInteger;
    schema:birthDate "2004-03-24"^^xsd:date;
    schema:familyName "Johnson";
    rdfs:label "Eric Johnson";
    schema:gender schema:Male;
    schema:givenName "Eric";
    schema:height "174"^^xsd:positiveInteger;
    schema:instructor ex:AryaParker, ex:PhillipMiller,ex:FrankJones ;
    ex:classmate ex:AliceDoe, ex:KatrinePhill.


# Phillip
ex:PhillipMiller a schema:Person;
    a wiki:Q121594;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "9500"^^xsd:positiveInteger;
    dbp:age "38"^^xsd:positiveInteger;
    schema:birthDate "1984-07-28"^^xsd:date;
    schema:familyName "Miller";
    rdfs:label "Phillip Miller";
    schema:gender schema:Male;
    schema:givenName "Phillip";
    schema:height "184"^^xsd:positiveInteger;
    schema:colleague ex:AryaParker, ex:FrankJones, ex:johnPitter, ex:codyModly, ex:JinzhouYang, ex:OzgeErten.

# Phil
ex:PhilJohnson a schema:Person;
    a dbe:AssistantProfessor;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "5200"^^xsd:positiveInteger;
    dbp:age "40"^^xsd:positiveInteger;
    schema:birthDate "1982-08-28"^^xsd:date;
    schema:familyName "Johnson";
    rdfs:label "Phil Johnson";
    schema:gender schema:Male;
    schema:givenName "Phil";
    schema:height "189"^^xsd:positiveInteger;
    schema:colleague ex:JohnBrown, ex:PhillipMiller.

# Patrick
ex:PatrickWilson a schema:Person;
    a dbe:AssistantProfessor;
    schema:employee schema:CollegeOrUniversity;
    schema:baseSalary "4700"^^xsd:positiveInteger;
    dbp:age "45"^^xsd:positiveInteger;
    schema:birthDate "1977-07-10"^^xsd:date;
    schema:familyName "Wilson";
    rdfs:label "Patrick Wilson";
    schema:gender schema:Male;
    schema:givenName "Patrick";
    schema:height "175"^^xsd:positiveInteger;
    schema:colleague ex:JohnBrown, ex:PhilJohnson.

# Elsa
ex:ElsaTaylor a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "21"^^xsd:positiveInteger;
    schema:birthDate "2001-03-08"^^xsd:date;
    schema:familyName "Taylor";
    rdfs:label "Elsa Taylor";
    schema:gender schema:Female;
    schema:givenName "Elsa";
    schema:height "155"^^xsd:positiveInteger;
    schema:instructor ex:JohnBrown, ex:PatrickWilson;
    ex:classmate ex:GeorgeAnderson, ex:SelenaGarcia, ex:HarryLee.

# Harry
ex:HarryLee a schema:Person;
    a dbe:Student ;
    dbo:student schema:CollegeOrUniversity;
    dbp:age "22"^^xsd:positiveInteger;
    schema:birthDate "2000-05-08"^^xsd:date;
    schema:familyName "Lee";
    rdfs:label "Harry Lee";
    schema:gender schema:Male;
    schema:givenName "Harry";
    schema:height "177"^^xsd:positiveInteger;
    schema:instructor ex:FrankJones, ex:PhilJohnson;
    ex:classmate ex:GeorgeAnderson, ex:ElsaTaylor, ex:KarenJones, ex:MelanieDavis .
"""

#Code

In [ ]:
# Execute the cell above and this cell to load the RDF graph and the schema.org vocabulary
from rdflib import Graph
g = Graph()
# g.parse("assignment2_dataset.ttl",format="ttl")
g.parse(data=university_rdf,format="ttl")
g.parse('https://schema.org/version/latest/schemaorg-current-https.ttl')

<Graph identifier=N544ea62279a641e48ca4c642564883fb (<class 'rdflib.graph.Graph'>)>

**1. List the top four oldest people in the graph from oldest to youngest**

In [ ]:
TA_query = """
PREFIX schema: <http://schema.org/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT ?Person ?age
WHERE {
    ?Person a schema:Person ;
              dbp:age ?age .
} 
ORDER BY DESC(?age)
LIMIT 4
"""
qres = g.query(TA_query)
for row in qres:
    print(f"{row.Person} {row.age}")

http://example.org/FrankJones 76
http://example.org/KatrinePhill 65
http://example.org/AryaParker 57
http://example.org/JohnBrown 53


**2. Identify the shortest student who has at least four classmates**


In [ ]:
TA_query = """
PREFIX schema:<http://schema.org/>
PREFIX dbo:<https://dbpedia.org/ontology/>
PREFIX ex:<http://example.org/>

SELECT ?Person ?height 
WHERE {
  {
   SELECT ?Person
   WHERE 
   {
    ?Person a schema:Person ;
              dbo:student ?student .
    ?Person ex:classmate ?classmate.
   }
   GROUP BY ?Person
   HAVING(COUNT(?classmate) >= 4)
   }
  ?Person schema:height ?height
}
ORDER BY ?height
LIMIT 1
"""
qres = g.query(TA_query)
for row in qres:
    print(f"{row.Person} {row.height}")

http://example.org/KatrinePhill 175


In [ ]:
TA_query = """
PREFIX schema:<http://schema.org/>
PREFIX dbo:<https://dbpedia.org/ontology/>
PREFIX ex:<http://example.org/>

SELECT ?Person ?height (COUNT(?classmate) AS ?countclassmate)
WHERE {
  ?Person a schema:Person ;
            dbo:student ?student .
  ?Person ex:classmate ?classmate .
  ?Person schema:height ?height .
}
GROUP BY ?Person ?height
HAVING(COUNT(?classmate) >= 4)
ORDER BY ASC(?height)
LIMIT 1
"""
qres = g.query(TA_query)
for row in qres:
    print(f"{row.Person} {row.height} {row.countclassmate}")

http://example.org/KatrinePhill 175 5


**3. List the teachers who have at least five colleagues and order them by highest salaries (in increasing order)?**

In [ ]:
TA_query = """
PREFIX dbe:<https://dbpedia.org/page/>
PREFIX dbo: <https://dbpedia.org/ontology/>
PREFIX schema: <http://schema.org/>

SELECT ?Person ?baseSalary
WHERE {
    ?Person a schema:Person ;
              schema:employee ?employee ;
              schema:baseSalary ?baseSalary .
    ?Person schema:colleague ?colleague.
}
GROUP BY ?Person ?baseSalary
HAVING(COUNT(?colleague) >= 5)
ORDER BY ?baseSalary
"""

qres = g.query(TA_query)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://example.org/AryaParker'), rdflib.term.Literal('5000', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#positiveInteger')))
(rdflib.term.URIRef('http://example.org/JohnBrown'), rdflib.term.Literal('5300', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#positiveInteger')))
(rdflib.term.URIRef('http://example.org/PhillipMiller'), rdflib.term.Literal('9500', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#positiveInteger')))


**4. Return the mean (average) age of male professors, and the mean age of female professors**

In [ ]:
TA_query = """
PREFIX schema: <http://schema.org/>
PREFIX dbp: <http://dbpedia.org/property/> 

SELECT ?gender (AVG(?age) AS ?avgage)
WHERE {
    ?Person a schema:Person ;
              schema:gender ?gender ;
              rdf:type ?type ;
              dbp:age ?age .
    ?type rdfs:subClassOf* dbe:Teacher .

} GROUP BY ?gender

"""

qres = g.query(TA_query)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://schema.org/Male'), rdflib.term.Literal('50.4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://schema.org/Female'), rdflib.term.Literal('57', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))


**5. For each employee, calculate their "net salary"**


In [ ]:
# For professior
TA_query = """
PREFIX wiki: <https://www.wikidata.org/wiki/>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?Person ?netSalary

WHERE {
    ?Person a schema:Person ;
              schema:employee ?employee ;
              schema:baseSalary ?baseSalary .
    BIND((?baseSalary - ?baseSalary*.04 - ?baseSalary*.06) AS ?netSalary)
}

"""

qres = g.query(TA_query)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://example.org/JohnBrown'), rdflib.term.Literal('4770.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://example.org/AryaParker'), rdflib.term.Literal('4500.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://example.org/FrankJones'), rdflib.term.Literal('3870.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://example.org/PhillipMiller'), rdflib.term.Literal('8550.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://example.org/PhilJohnson'), rdflib.term.Literal('4680.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))
(rdflib.term.URIRef('http://example.org/PatrickWilson'), rdflib.term.Literal('4230.00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))


**6. List all people with their first name or family name starting with the letter F, E, or A**

In [ ]:
TA_query = """
PREFIX schema: <http://schema.org/>
PREFIX dbp: <http://dbpedia.org/property/>

SELECT DISTINCT ?Person ?name
WHERE {
    ?Person a schema:Person ;
      schema:familyName|schema:givenName ?name .
    VALUES ?filter_letter { "f" "e" "a" }
    FILTER(strstarts(lcase(?name), ?filter_letter))
}
"""
qres = g.query(TA_query)
for row in qres:
    print(f"{row.Person} {row.name}")

http://example.org/AliceDoe Alice
http://example.org/FrankJones Frank
http://example.org/GeorgeAnderson Anderson
http://example.org/EricJohnson Eric
http://example.org/ElsaTaylor Elsa


**7. List all `dbe:Researcher` using inference**

Your query should also returns subclasses of `dbe:Researcher`, such as PhD Candidates and AssistantResearcher, without directly using their URI (use the `rdfs:subClassOf` properties)

**Hint: Use inference**




In [ ]:
TA_query = """
PREFIX dbo: <https://dbpedia.org/ontology/>
PREFIX dbe: <https://dbpedia.org/page/>
PREFIX schema:<http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT  ?researcher
WHERE {
   ?researcher rdf:type/(rdfs:subClassOf|owl:equivalentClass)* dbe:Researcher .
}

"""

qres = g.query(TA_query)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://example.org/KatrinePhill'),)
(rdflib.term.URIRef('http://example.org/AliceDoe'),)
(rdflib.term.URIRef('http://example.org/JohnBrown'),)
(rdflib.term.URIRef('http://example.org/PhillipMiller'),)
(rdflib.term.URIRef('http://example.org/AryaParker'),)
(rdflib.term.URIRef('http://example.org/FrankJones'),)
(rdflib.term.URIRef('http://example.org/PhilJohnson'),)
(rdflib.term.URIRef('http://example.org/PatrickWilson'),)


**8. Create missing relations when `:isSupervisedBy` is defined in one direction but not `:supervises` in the other direction. For example, if john `:isSupervisedBy` mary, then we know that mary `:supervises` john**

Requirements: you should use a CONSTRUCT query to build the missing triples


In [ ]:
TA_query = """
PREFIX dbo: <https://dbpedia.org/ontology/>
PREFIX dbe: <https://dbpedia.org/page/>
PREFIX schema:<http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX : <http://example.org/voc#>

PREFIX schema: <http://schema.org/>
CONSTRUCT {
    ?prof :supervises ?student .
}
WHERE {
  ?student :isSupervisedBy ?prof .
  MINUS {
    ?prof :supervises ?student .
  }
}
"""
qres = g.query(TA_query)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://example.org/JohnBrown'), rdflib.term.URIRef('http://example.org/voc#supervises'), rdflib.term.URIRef('http://example.org/AliceDoe'))


**9. Count the number of men and women per `rdf:type` in the graph (aka. the `a` relation)**

In [ ]:
TA_query = """
PREFIX dbo: <https://dbpedia.org/ontology/>
PREFIX dbe: <https://dbpedia.org/page/>
PREFIX schema:<http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX : <http://example.org/voc#>


PREFIX schema: <http://schema.org/>
SELECT ?type ?gender (count(?person) as ?count)
WHERE {
    ?person a ?type ;
        schema:gender ?gender .
}
GROUP BY ?type ?gender
ORDER BY DESC(?count)
"""
qres = g.query(TA_query)
for row in qres:
    print(row)


(rdflib.term.URIRef('http://schema.org/Person'), rdflib.term.URIRef('http://schema.org/Male'), rdflib.term.Literal('8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://schema.org/Person'), rdflib.term.URIRef('http://schema.org/Female'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://dbpedia.org/page/Student'), rdflib.term.URIRef('http://schema.org/Female'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://dbpedia.org/page/AssistantProfessor'), rdflib.term.URIRef('http://schema.org/Male'), rdflib.term.Literal('3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://dbpedia.org/page/Student'), rdflib.term.URIRef('http://schema.org/Male'), rdflib.term.Literal('3', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int

**10. Write a federated query to Wikidata to retrieve the inception date of Maastricht University**

In [ ]:
TA_query = """
PREFIX dbo: <https://dbpedia.org/ontology/>
PREFIX dbe: <https://dbpedia.org/page/>
PREFIX schema:<http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX : <http://example.org/voc#>
PREFIX wiki: <http://www.wikidata.org/entity/>
PREFIX wdp: <http://www.wikidata.org/prop/>
PREFIX wdps: <http://www.wikidata.org/prop/statement/>

SELECT DISTINCT ?date
WHERE {
  ex:AryaParker schema:employee ?uni.
    SERVICE <https://query.wikidata.org/sparql> {
      # Maastricht University inception date
      ?uni wdp:P571 ?date .
    }
} LIMIT 1
"""
qres = g.query(TA_query)
for row in qres:
    print(row.date)
